In [10]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

import torch

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


In [11]:
# Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
model_method = 'GRU_rg'

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

In [12]:
# App_name
app_name = "Hs2SwineFarmWithWeatherTime" # "Hs2SwineFarmWithWeatherTime", "energy"
step ='train'

# 2. Training 

## 2-1. Data selection

In [13]:
get_ms_list = db_client.measurement_list(bucket_name)
get_ms_list

['classification_actionPattern_testX_cleanLevel0',
 'classification_actionPattern_testy_cleanLevel0',
 'classification_actionPattern_trainX_cleanLevel0',
 'classification_actionPattern_trainy_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel4',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel4',
 'forecasting_strawberryOpen_test_cleanLevel0',
 'forecasting_strawberryOpen_test_cleanLevel4',
 'forecasting_strawberryOpen_train_cleanLevel0',
 'forecasting_strawberryOpen_train_cleanLevel4',
 'regression_energy_testX_cleanLevel0',
 'regression_energy_testX_cleanLevel4',
 'regression_energy_testy_cleanLevel0',
 'regres

In [14]:
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

['forecasting_strawberryOpen',
 'regression_energy',
 'forecasting_Hs2SwineFarmWithWeatherTime',
 'forecasting_gunwiStrawberryWeather',
 'classification_actionPattern']

In [15]:
data_clean_level = 0 
if app_name == "energy":
    model_purpose = 'regression'
    dataset_name = model_purpose+'_' +app_name
    data_name_X = dataset_name+'_'+step+'X_cleanLevel'+str(data_clean_level)
    data_name_y = dataset_name+'_'+step+'y_cleanLevel'+str(data_clean_level)
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="window_split"
    
elif app_name == "Hs2SwineFarmWithWeatherTime":
    model_purpose = 'forecasting' 
    dataset_name = model_purpose+'_' +app_name
    data_name_X = dataset_name+'_'+step+'_cleanLevel'+str(data_clean_level)
    data_name_y = None
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'

## X Data Ingestion & scaling

In [16]:
# Data Ingestion
from Clust.clust.data import data_interface
ingestion_param = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
data_X = data_interface.get_data_result("ms_all", db_client, ingestion_param)
data_X = data_X[feature_X_list]

# Data Scaling
from Clust.clust.ML.tool import scaler
scalerParam='scale'
scaleMethod='minmax'

scalerRootPath_X = os.path.join(scalerPath, data_name_X)
dataX_scaled, X_scalerFilePath = scaler.get_data_scaler(scalerParam, scalerRootPath_X, data_X, scaleMethod)                                                                      

['Temperature', 'out_temp', 'sin_hour']
Make New scaler File


## Y Data Ingestion & Scaling

In [17]:
if data_name_y:
    ingestion_param = {
        "bucket_name" : bucket_name,
        'ms_name' : data_name_y,
        'feature_list' : feature_y_list                             
    }
    data_y = data_interface.get_data_result("ms_all", db_client, ingestion_param)
    
else: # y 가 없다면 데이터를 만들어야 함
    data_y = data_X[feature_y_list]
    data_name_y = dataset_name+'_'+step+'y_cleanLevel'+str(data_clean_level)
 
scalerRootPath_y = os.path.join(scalerPath, data_name_y)
datay_scaled, y_scalerFilePath = scaler.get_data_scaler(scalerParam, scalerRootPath_y, data_y, scaleMethod)

['Temperature']
Make New scaler File


## X Feature Selection (column 별로 체크, 문제가 많은 컬럼은 삭제)

In [22]:
# Clean
model_clean = True
nan_process_info = {'type':'num', 'ConsecutiveNanLimit':1000000, 'totalNaNLimit':10000000}
# 이부분 신경 써야함

from Clust.clust.quality.NaN import cleanData
CMS = cleanData.CleanData()

# Feature Selection 넣을 경우 이 모듈에 코딩
dataX_scaled = CMS.get_cleanData_by_removing_column(dataX_scaled, nan_process_info) 

3 ---> 3


In [19]:
if model_clean:
    # model_clean 파라미터를 쓰고 싶은 경우, 아래에서 체크
    # interpolation을 하지 않고 그 데이터를 삭제함
    pass
else:
    dataX_scaled = dataX_scaled.interpolate(method='linear')
    datay_scaled = datay_scaled.interpolate(method='linear')

## Data Split

In [23]:
# Split
from Clust.clust.transformation.purpose import machineLearning as ML
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
split_ratio = 0.8
# TODO 데이터 나뉘는 부분 추가로 작성된 것 지수님에게 물어봐야 함

if split_mode =='window_split':
    from datetime import timedelta 
    # define window size by clust structure
    first_date = dataX_scaled.index[0]
    window_size = dataX_scaled.loc[first_date:first_date + timedelta(days =0, hours=23, minutes=59, seconds=59)].shape[0]
    print(window_size)
    train_x, val_x = ML.split_data_by_ratio(dataX_scaled, split_ratio, split_mode, window_size)
else:
    window_size =None
    train_x, val_x = ML.split_data_by_ratio(dataX_scaled, split_ratio, None, None)
    
train_y, val_y = ML.split_data_by_ratio(datay_scaled, split_ratio, None, None)

## Data Transformation for training

In [26]:
import numpy as np
if split_mode =='window_split':
    from Clust.clust.transformation.type import DFToNPArray
    transformParameter = {
        'window_num':window_size
    }
    train_X_array = DFToNPArray.trans_DF_to_NP_by_windowNum(train_x, transformParameter['window_num'])
    val_X_array= DFToNPArray.trans_DF_to_NP_by_windowNum (val_x, transformParameter['window_num'])
    train_y_array = train_y.values.reshape(-1)
    val_y_array = val_y.values.reshape(-1)
    
elif split_mode == 'step_split':
    transformParameter = {
        'future_step': 2,
        'past_step': 24
    }
    train_X_array, train_y_array = ML.trans_by_step_info(train_x, train_y, transformParameter)
    val_X_array, val_y_array = ML.trans_by_step_info(val_x, val_y, transformParameter)

Original Length: 8516 Final Length: 8326 NaN Length: 190
Original Length: 2129 Final Length: 2103 NaN Length: 26


## Set Model Parameters & Train Parameters

In [ ]:
# RNN models (RNN, LSTM, GRU) parameters
seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }
    
trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}

In [ ]:
# 3. 모델을 저장할 파일 패스를 생성한다.
model_name = None
if model_name is None:
    model_name = dataset_name + '_' + model_method + '_modelCleanLevel' + str(model_clean)
else:
    pass

from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, data_name_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

## 2-5 Training 

In [ ]:
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML

rml = RML()
rml.set_param(trainParameter)
rml.set_model(model_method, modelParameter)
rml.set_data(train_X_array, train_y_array, val_X_array, val_y_array, )
rml.train()
rml.save_best_model(modelFilePath)

## 2-6 Save MetaData 

In [ ]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
modelTags =["model_tag_example"]
trainDataType = "timeseries"

modelInfoMeta ={
    "trainDataInfo":data_meta,
    "modelName":model_name,
    "featureList":feature_X_list,
    "target": feature_y_list,
    "trainDataType":trainDataType,
    "modelPurpose":model_purpose,
    "modelMethod":model_method,
    "modelTags":modelTags,
    "modelCleanLevel":model_clean,
    "trainParameter": trainParameter,
    "modelParameter": modelParameter,
    "transformParameter":transformParameter,
    "scalerParam":scalerParam,
    "trainDataName":[data_name_X, data_name_y], 

    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":modelFilePath
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath      
        }
    }
}

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)